In [20]:
import json
import os
from pprint import pprint


import tweepy
import requests

headers = {
    'X-API-Key': os.environ["PP_API_KEY"],
}

### get members

In [25]:
member_list = []

with open('../data/member_data.json', 'r') as ofile:
    member_list = json.loads(ofile.read())
ofile.close()

### just get member ids

In [26]:
member_ids = []

for member in member_list:
    member_ids.append(member["id"])

### get most recent bills

In [27]:
future_cong = True
this_cong = 123

while future_cong == True:
    resp_json = json.loads(requests.get("https://api.propublica.org/congress/v1/"+ str(this_cong) +"/house/bills/introduced.json", headers=headers).text)
    if resp_json["status"] == "500":
        this_cong = this_cong - 1
    else:
        future_cong = False


### filter to just this state bills

In [32]:
this_state_bills = []

for bill in resp_results[0]['bills']:
    if bill["sponsor_id"] in member_ids:
        this_state_bills.append(bill)

In [37]:
this_state_bills

[{'bill_id': 'hr7252-115',
  'bill_slug': 'hr7252',
  'bill_type': 'hr',
  'number': 'H.R.7252',
  'bill_uri': 'https://api.propublica.org/congress/v1/115/bills/hr7252.json',
  'title': 'To direct the Attorney General to enter into an agreement with the National Academies to conduct a study to develop guidelines, best practices, and examples for congressional redistricting.',
  'short_title': 'To direct the Attorney General to enter into an agreement with the National Academies to conduct a study to develop guidelines, best practices, and examples for congressional redistricting.',
  'sponsor_title': 'Rep.',
  'sponsor_id': 'M001166',
  'sponsor_name': 'Jerry McNerney',
  'sponsor_state': 'CA',
  'sponsor_party': 'D',
  'sponsor_uri': 'https://api.propublica.org/congress/v1/members/M001166.json',
  'gpo_pdf_uri': None,
  'congressdotgov_url': 'https://www.congress.gov/bill/115th-congress/house-bill/7252',
  'govtrack_url': 'https://www.govtrack.us/congress/bills/115/hr7252',
  'introdu

### craft tweets

In [46]:
congress_conversions = {'115': '115th', '116': '116th', '117': '117th', '118': '118th', '119': '119th', '120': '120th', '121': '121st', '122': '122nd', '123': '123rd'}

max_tweet_len = 280
link_len = 23
allowed_text_length = max_tweet_len - link_len - 5

for bill in this_state_bills:
    member_dict = [x for x in member_list if x["id"] == bill["sponsor_id"]][0]
    t_text = ""
    
    try:
        full_name = ".@" + member_dict["twitter_id"] + " (" + member_dict["party"] + ")" 
    except:
        full_name = member_dict["role"][0:3] + ". " + member_dict["name"] + " (" + member_dict["party"] + ")" 
        
        
    t_text = t_text + full_name + " introduced " + bill["number"]
    t_text = t_text + ", a bill to " + bill["short_title"].replace('To ', '')
    
    
    if len(t_text) > allowed_text_length:
        t_text = t_text[0:allowed_text_length] + "... "
        
        
    congress_ord = congress_conversions[str(this_cong)]
    bill_nums =  [x for x in list(bill["number"]) if x.isdigit()]
    bill_num = "".join(bill_nums)
    bill_chamber = "senate"
    if "H" in bill['number'].upper():
        bill_chamber = "house"
    congress_gov_url = "https://www.congress.gov/bill/"+ congress_ord +"-congress/" + bill_chamber + "-bill/"+ bill_num
    
    t_text = t_text + " " + congress_gov_url
    
    print(t_text)
    
    bill["t_text"] = t_text

.@RepMcNerney (D) introduced H.R.7252, a bill to direct the Attorney General to enter into an agreement with the National Academies to conduct a study to develop guidelines, best practices, and examples for congressional redistricting. https://www.congress.gov/bill/115th-congress/house-bill/7252
